In [1]:
import numpy as np, pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("hindi-train.csv")

In [4]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [5]:
count_length()
df.tail(10)

,headline,Label,word_count
35541,माधुरी दीक्षित ने मधुबाला के हिट गाने पर किया ...,Entertainment,22
33201,क्रूज रेव पार्टी मामला: आर्यन खान के पास से भी...,Entertainment,19
14696,अंतिम यात्रा में भी पार्टनर का साथ नहीं छोड़ रह...,Tech,14
33867,"वेडिंग फोटोशूट करा रहे थे दूल्हा-दुल्हन, कीचड़ ...",Entertainment,13
18898,बिना टिकट यात्रा करने वाले को चलती ट्रेन से ला...,Business,18
31019,"अफ्रीकी दोस्तों ने गाया मोहम्मद रफी का गाना, द...",Entertainment,13
30280,अंतिम यात्रा में भी पार्टनर का साथ नहीं छोड़ रह...,Entertainment,14
6637,टेनिस : फ्रेंच ओपन के सेमीफाइनल में भिड़ सकते ह...,Sports,13
35343,संजय दत्त के जन्मदिन पर रिलीज हुआ 'केजीएफ चैप्...,Entertainment,17
23720,अमित शाह पर निशाना साधने के लिए 'शायर' बने राह...,Politics,15


In [6]:
import re
def processText(text):
    text = text.lower()
    text = re.sub("[a-zA-z0-9]",' ',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text

In [7]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])

In [8]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['headline'] = df['headline'].apply(lambda x: tokenization(x))

In [9]:
for i in range(len(df)):
    df['headline'][i] = [s.replace("\n", "") for s in df['headline'][i]]

In [10]:
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
to_be_removed = stopwords_hi + punctuations + stopwords_en

In [11]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (to_be_removed)]
count_length()
df.tail(10)

,headline,Label,word_count
35541,"[माधुरी, दीक्षित, मधुबाला, हिट, गाने, खूबसूरत,...",Entertainment,13
33201,"[क्रूज, रेव, पार्टी, मामला, आर्यन, खान, पास, ड...",Entertainment,13
14696,"[अंतिम, यात्रा, पार्टनर, छोड़, मोर, देखें, इमोश...",Tech,8
33867,"[वेडिंग, फोटोशूट, करा, दूल्हा, दुल्हन, कीचड़, ध...",Entertainment,8
18898,"[बिना, टिकट, यात्रा, चलती, ट्रेन, लात, मार, पु...",Business,10
31019,"[अफ्रीकी, दोस्तों, गाया, मोहम्मद, रफी, गाना, द...",Entertainment,10
30280,"[अंतिम, यात्रा, पार्टनर, छोड़, मोर, देखें, इमोश...",Entertainment,8
6637,"[टेनिस, फ्रेंच, ओपन, सेमीफाइनल, भिड़, नडाल, फेडरर]",Sports,7
35343,"[संजय, दत्त, जन्मदिन, रिलीज, केजीएफ, चैप्टर, न...",Entertainment,11
23720,"[अमित, शाह, निशाना, साधने, शायर, बने, राहुल, ग...",Politics,11


In [12]:
import collections
from collections import Counter
corpus_list =[]
for i in range(len(df)):
    corpus_list +=df['headline'][i]
counter=collections.Counter(corpus_list)

In [13]:
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (least_common)]

In [14]:
df

,headline,Label,word_count
13020,"[फोन, डाउनलोड, वीडियो, ऐप, जरूरत]",Tech,5
29809,"[लोकपाल, दोहरा, मानदंड, मोदी, सरकार, कांग्रेस]",Politics,6
26445,"[राजस्थान, एमपी, छत्तीसगढ़, कांग्रेस, मेगा, शो...",Politics,11
36138,"[ऑफर, दिलीप, जोशी, एक्टिंग, छोड़ने, रोका, साल,...",Entertainment,9
22190,"[आईजीएल, पेट्रोनेट, हिस्सेदारी, बेच, सकती, बीप...",Business,6
...,...,...,...
31019,"[अफ्रीकी, दोस्तों, गाया, मोहम्मद, रफी, गाना, द...",Entertainment,10
30280,"[अंतिम, यात्रा, पार्टनर, छोड़, मोर, देखें, इमोश...",Entertainment,8
6637,"[टेनिस, फ्रेंच, ओपन, सेमीफाइनल, भिड़, नडाल, फेडरर]",Sports,7
35343,"[संजय, दत्त, जन्मदिन, रिलीज, केजीएफ, चैप्टर, न...",Entertainment,11


In [15]:
df['headline'] = df['headline'].str.join(" ")
df.tail(10)

,headline,Label,word_count
35541,माधुरी दीक्षित मधुबाला हिट गाने खूबसूरत सा डां...,Entertainment,13
33201,क्रूज रेव पार्टी मामला आर्यन खान पास ड्रग्स बर...,Entertainment,13
14696,अंतिम यात्रा पार्टनर छोड़ मोर देखें इमोशनल वीडियो,Tech,8
33867,वेडिंग फोटोशूट करा दूल्हा दुल्हन कीचड़ धड़ाम गिरे,Entertainment,8
18898,बिना टिकट यात्रा चलती ट्रेन लात मार पुलिसवाला ...,Business,10
31019,अफ्रीकी दोस्तों गाया मोहम्मद रफी गाना देखकर इम...,Entertainment,10
30280,अंतिम यात्रा पार्टनर छोड़ मोर देखें इमोशनल वीडियो,Entertainment,8
6637,टेनिस फ्रेंच ओपन सेमीफाइनल भिड़ नडाल फेडरर,Sports,7
35343,संजय दत्त जन्मदिन रिलीज केजीएफ चैप्टर नया पोस्...,Entertainment,11
23720,"अमित शाह निशाना साधने शायर बने राहुल गांधी "" ख...",Politics,11


In [15]:
train, test = train_test_split(df,test_size=0.3,random_state=42)

In [16]:
train.shape

(26250, 3)

In [17]:
test.shape

(11250, 3)

In [18]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB(alpha=0.1))
model.fit(train.headline, train.Label)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB(alpha=0.1))])

In [19]:
predicted_categories = model.predict(test.headline)

In [20]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("The accuracy is {}".format(accuracy_score(test.Label, predicted_categories)*100))

The accuracy is 74.78222222222223


In [21]:
import joblib
import os
loaded_model = joblib.dump(model, 'hin_nb.pkl')

In [22]:
mat = confusion_matrix(test.Label, predicted_categories)
mat

array([[1763,   40,  187,   75,  121],
       [  92, 1394,  226,   96,  497],
       [ 100,   85, 1900,   97,   24],
       [  82,  193,  228, 1514,  277],
       [ 167,  174,   38,   38, 1842]], dtype=int64)

In [23]:
report = classification_report(test.Label, predicted_categories)
print(report)

               precision    recall  f1-score   support

     Business       0.80      0.81      0.80      2186
Entertainment       0.74      0.60      0.67      2305
     Politics       0.74      0.86      0.79      2206
       Sports       0.83      0.66      0.74      2294
         Tech       0.67      0.82      0.73      2259

     accuracy                           0.75     11250
    macro avg       0.75      0.75      0.75     11250
 weighted avg       0.75      0.75      0.75     11250



In [24]:
#business
simple_test1 = ["सेंसेक्स 1129 पॉइंट्स लुढ़का, रिलायंस का शेयर 4.5% टूटा, मार्केट कैप 2 घंटे में 7.88 लाख करोड़ घटाााा"]
pred1 = model.predict(simple_test1)
pred1[0]

'Business'

In [25]:
#entertainment
simple_test2 = ["पलक तिवारी की ग्लैमरस फोटोज, सलमान की Ex गर्लफ्रेंड ने किया कमेंट"]
pred2 = model.predict(simple_test2)
pred2[0]

'Entertainment'

In [26]:
#political
simple_test3 = ["पश्चिमी यूपी से पूर्वांचल, अवध से बुंदेलखंड तक, कैसे कमल की फसल को खाद दे रहे पीएम?"]
pred3 = model.predict(simple_test3)
pred3[0]

'Politics'

In [27]:
#sports
simple_test4 = ["भारत पहुंची पाकिस्तान की टीम, 24 नवंबर से भुवनेश्वर में खेला जाएगा वर्ल्ड कपाा"]
pred4 = model.predict(simple_test4)
pred4[0]

'Sports'

In [28]:
#tech
simple_test5 = ["Jio ने लॉन्‍च किए दो प्रीपेड रिचार्ज प्लान, मिलेगा Disney+ Hotstar प्रीमियम सब्सक्रिप्शन, जानें डिटेलाा"]
pred5 = model.predict(simple_test5)
pred5[0]

'Tech'